In [1]:
!pip install pyLDAvis --quiet
!pip install chart_studio --quiet


     |████████████████████████████████| 1.6MB 6.1MB/s 
     |████████████████████████████████| 71kB 3.6MB/s 


In [2]:
import pandas as pd
import numpy as np
import time
import re
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
import gensim
from spacy.tokenizer import Tokenizer
import gensim.corpora as corpora
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint
from gensim.models.coherencemodel import CoherenceModel
import plotly.express as px
import pyLDAvis.gensim
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
from operator import itemgetter
from ipywidgets import interact
import tqdm
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets


# supress warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will load in our preprocessed tweets from our [data cleaning notebook](https://github.com/tarrantcarter/Final_Capstone/blob/main/Modern_Motivation_Data_Cleaning_Feature_Engineering.ipynb). The csv can be found [here](https://drive.google.com/file/d/1-d-61YuocweY0F3rSnf-idL1ixbuKq4H/view?usp=sharing). 

In [4]:
# load in cleaned tweets from data cleaning notebook
tweets_cleaned = pd.read_json("/content/drive/MyDrive/Data/NLP_Capstone/motivational_tweets_cleaned.json")

In [5]:
tweets_cleaned.head()

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,2021-01-17 22:13:17,LewisHowes,Know this. Everything is happening for a reaso...,know happen reason favor betterment future pai...,"[know, happen, reason, favor, betterment, futu...",know_happen_reason favor betterment future pai...,"[know_happen_reason, favor, betterment, future...","[reason, favor, betterment, future, pain, feel...","[know, reason, favor, betterment, future, pain...",[know_happen_reason],know_happen_reason
61,2021-01-15 15:28:06,LewisHowes,Protect your inner peace at all costs. Create ...,protect inner peace cost create daily practice...,"[protect, inner, peace, cost, create, daily, p...",protect inner_peace cost create daily_practice...,"[protect, inner_peace, cost, create, daily_pra...","[peace, cost, practice, communicate, stress, d...","[peace, cost, practice, communicate, stress, d...","[inner_peace, daily_practice]",inner_peace daily_practice
161,2021-01-07 16:00:29,LewisHowes,Always remember to ask for exactly what you wa...,remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...",remember ask exactly want ask love good health...,"[remember, ask, exactly, want, ask, love, good...","[health, abundance, peace, ask, wisdom, creati...","[remember, health, abundance, peace, ask, wisd...","[health_abundance, ask_wisdom]",health_abundance ask_wisdom
274,2021-01-01 02:44:46,LewisHowes,This will be your greatest year ever. All your...,great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...",great year work start pay earn happy love deep...,"[great, year, work, start, pay, earn, happy, l...","[year, work, start, earn, love, embrace, fear,...","[year, work, start, pay, earn, love, embrace, ...",[massive_action],massive_action
317,2020-12-30 01:38:35,LewisHowes,Be grateful for the breakdown this year. It's ...,grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...",grateful breakdown year set massive breakthrou...,"[grateful, breakdown, year, set, massive, brea...","[year, breakthrough, money, mission, get, rela...","[year, set, breakthrough, money, come, mission...",[real_friend],real_friend


In [6]:
tweets_cleaned.applymap(type)

,date,user_name,content,content_preprocessed,unigram_tokens,ngrams,ngram_tokens,nouns_only,nouns_verbs,bigrams_trigrams,bigrams_trigrams_strings
44,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
61,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
161,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
274,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
317,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...
977598,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977599,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977606,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>
977608,<class 'pandas._libs.tslibs.timestamps.Timesta...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>,<class 'str'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'list'>,<class 'str'>


In [7]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 621449 entries, 44 to 977610
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      621449 non-null  datetime64[ns]
 1   user_name                 621449 non-null  object        
 2   content                   621449 non-null  object        
 3   content_preprocessed      621449 non-null  object        
 4   unigram_tokens            621449 non-null  object        
 5   ngrams                    621449 non-null  object        
 6   ngram_tokens              621449 non-null  object        
 7   nouns_only                621449 non-null  object        
 8   nouns_verbs               621449 non-null  object        
 9   bigrams_trigrams          621449 non-null  object        
 10  bigrams_trigrams_strings  621449 non-null  object        
dtypes: datetime64[ns](1), object(10)
memory usage: 56.9+ MB


In [8]:
tweets_cleaned.shape

(621449, 11)

# Topic Modeling

In [9]:
# create dictionary
id2word = corpora.Dictionary(tweets_cleaned['nouns_verbs'])
# create texts corpus
texts = tweets_cleaned['nouns_verbs']
# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]
# print first 30 tuples from corpus
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]


In [10]:
# number of topics
num_topics = 10
# build LDA model
base_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# # print the keyword in the 10 topics
# pprint(base_model.print_topics())
# doc_lda = base_model[corpus]

In [11]:
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
time life look problem try churchill winston robert day solve

------ Topic 1 ------
life way power person create failure success happen find jim

------ Topic 2 ------
change thought dyer mind body peace feel franklin result lincoln

------ Topic 3 ------
dyer day think control george moment ability miracle define life

------ Topic 4 ------
man opportunity eye ralph napoleon person character word care time

------ Topic 5 ------
world life go abundance habit think henry hold build jung

------ Topic 6 ------
thing dyer amp today life peale energy experience ask learn

------ Topic 7 ------
come work let heart place step love courage respect get

------ Topic 8 ------
choice dream reality act desire edison matter woman stop presence

------ Topic 9 ------
love einstein strength john fall treat judge proverb trust albert



In [12]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=tweets_cleaned['nouns_verbs'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -7.0271661492609825

Coherence Score:  0.5652556573583163


In [13]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(base_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.081715 -0.133343       1        1  11.502454
6      0.253952  0.031142       2        1  10.965268
4     -0.136100 -0.197542       3        1  10.561949
2      0.216165 -0.106330       4        1  10.521989
0     -0.084703 -0.017648       5        1  10.071324
9     -0.166400 -0.207211       6        1   9.963172
1     -0.050919  0.180100       7        1   9.825359
8     -0.130878  0.186390       8        1   9.552783
3      0.285625  0.006803       9        1   9.301172
5     -0.105026  0.257638      10        1   7.734529, topic_info=            Term           Freq          Total Category  logprob  loglift
1463        dyer  112346.000000  112346.000000  Default  30.0000  30.0000
239          man   34889.000000   34889.000000  Default  29.0000  29.0000
72         thing   35276.000000   35276.000000  Default  28.0000  28.0000
159        world   24529.000000   24529.000000  Default  27.0000  27.0000
119         time   35716.000000   35716.000000  Default  26.0000  26.0000
...          ...            ...            ...      ...      ...      ...
13          pain    1948.716694    2203.477774  Topic10  -4.6926   2.4366
103        think    4641.525216    9168.830044  Topic10  -3.8247   1.8787
758   reputation    1917.683025    2210.466739  Topic10  -4.7087   2.4174
78          life    6413.772147   64609.428397  Topic10  -3.5013   0.2496
143          job    1968.526237    2940.362107  Topic10  -4.6825   2.1582

[555 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
175       1  0.000252  ability
175       2  0.000757  ability
175       3  0.204230  ability
175       4  0.000252  ability
175       6  0.001262  ability
...     ...       ...      ...
159       9  0.000082    world
159      10  0.985268    world
597       1  0.000564    worry
597       3  0.999397    worry
3490      3  0.999775     yogi

[1113 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 5, 3, 1, 10, 2, 9, 4, 6])

In [14]:
get_document_topics = [base_model.get_document_topics(item) for item in corpus]

In [15]:
len(get_document_topics)

621449

In [16]:
get_document_topics[:20]

[[(1, 0.07101079),
  (2, 0.13581105),
  (4, 0.13433206),
  (5, 0.18648537),
  (6, 0.18274525),
  (8, 0.09069367),
  (9, 0.18126833)],
 [(0, 0.102748096),
  (1, 0.29417434),
  (2, 0.30613267),
  (4, 0.15891053),
  (6, 0.0995677)],
 [(2, 0.13333443),
  (3, 0.09001121),
  (5, 0.092044435),
  (6, 0.25910264),
  (7, 0.09363634),
  (8, 0.21832901),
  (9, 0.09046407)],
 [(0, 0.08458617),
  (2, 0.23480786),
  (4, 0.10348957),
  (6, 0.23976971),
  (7, 0.09783606),
  (8, 0.088557445),
  (9, 0.1278747)],
 [(1, 0.09384663),
  (3, 0.17705303),
  (6, 0.10579556),
  (7, 0.34577695),
  (8, 0.23584943)],
 [(1, 0.13708286),
  (2, 0.34397542),
  (4, 0.0979844),
  (5, 0.072905205),
  (6, 0.20303252),
  (8, 0.047579274),
  (9, 0.07979118)],
 [(2, 0.1618415), (6, 0.21317543), (8, 0.10222257), (9, 0.4727558)],
 [(0, 0.02),
  (1, 0.21989796),
  (2, 0.24108222),
  (3, 0.02),
  (4, 0.02),
  (5, 0.22),
  (6, 0.19901904),
  (7, 0.02),
  (8, 0.02),
  (9, 0.020000769)],
 [(2, 0.10656946),
  (5, 0.226691),
  (6, 0.2

In [17]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           alpha=a,
                                           eta=b,
                                           random_state=222)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [18]:
# start preprocess runtime
start_time = time.time() 

# Topics range
min_topics = 3
max_topics = 10
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = [.01,.1,.5,1,'symmetric','asymmetric']
# Beta parameter
beta = [.01,.1,.5,1,'symmetric']

# model results dict
model_results = {'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=210)

    # iterate through number of topics
    for k in topics_range:
        # iterate through alpha values
        for a in alpha:
            # iterare through beta values
            for b in beta:
                # get the coherence score for the given parameters
                cv = compute_coherence_values(corpus=corpus, dictionary=id2word, 
                                              k=k, a=a, b=b)
                # Save the model results
                model_results['Topics'].append(k)
                model_results['Alpha'].append(a)
                model_results['Beta'].append(b)
                model_results['Coherence'].append(cv)
                
                pbar.update(1)
    lda_tuning_results = pd.DataFrame(model_results)                
    lda_tuning_results.to_csv('/content/drive/MyDrive/Data/NLP_Capstone/lda_tuning_results.csv', index=False)
    pbar.close()

# print preprocess runtime
print(time.strftime(f'%H hours, %M minutes, %S seconds', time.gmtime(time.time() - start_time)))

100%|██████████| 210/210 [4:21:43<00:00, 74.78s/it]

04 hours, 21 minutes, 43 seconds


In [19]:
lda_tuning_results.sort_values('Coherence',ascending=False).head(30)

,Topics,Alpha,Beta,Coherence
161,8,0.5,0.1,0.592711
160,8,0.5,0.01,0.591234
190,9,0.5,0.01,0.590173
164,8,0.5,symmetric,0.588483
191,9,0.5,0.1,0.586364
130,7,0.5,0.01,0.583838
133,7,0.5,1,0.581137
163,8,0.5,1,0.579718
207,9,asymmetric,0.5,0.578841
205,9,asymmetric,0.01,0.577674


In [20]:
# number optimal of topics
num_topics = 8
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
man power mind act matter happiness opportunity churchill happen winston

------ Topic 1 ------
thing think proverb dream john man today judge robert trust

------ Topic 2 ------
change let desire decision bind tzu forget dog water harm

------ Topic 3 ------
dyer way amp thought success einstein strength problem action reality

------ Topic 4 ------
life peace amp body respect control ask napoleon presence woman

------ Topic 5 ------
heart come man try lincoln courage succeed abraham learn lama

------ Topic 6 ------
time world day choice create fall lie henry experience george

------ Topic 7 ------
love person look get feel work place step franklin eye



In [21]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['nouns_verbs'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.922985491470671

Coherence Score:  0.5949793728214159


In [22]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.294870  0.097503       1        1  13.896215
1     -0.197722  0.145241       2        1  13.380348
0     -0.158345 -0.082232       3        1  12.807796
4      0.219181  0.089010       4        1  12.753820
7      0.024154 -0.103180       5        1  12.466873
5     -0.186974  0.129864       6        1  11.797444
6     -0.008413  0.098364       7        1  11.746143
2      0.013249 -0.374570       8        1  11.151361, topic_info=         Term           Freq          Total Category  logprob  loglift
1463     dyer  106390.000000  106390.000000  Default  30.0000  30.0000
78       life   58753.000000   58753.000000  Default  29.0000  29.0000
119      time   39889.000000   39889.000000  Default  28.0000  28.0000
72      thing   37571.000000   37571.000000  Default  27.0000  27.0000
58     change   24800.000000   24800.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
113      meet    2405.998502    2476.873088   Topic8  -4.8477   2.1646
13       pain    2145.545855    2180.491854   Topic8  -4.9623   2.1775
1027   loving    3257.326266    3973.810524   Topic8  -4.5447   1.9948
175   ability    3258.906339    5713.465146   Topic8  -4.5443   1.6322
354     power    3654.794009   15463.566781   Topic8  -4.4296   0.7512

[505 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
175       1  0.012077  ability
175       2  0.000525  ability
175       3  0.297893  ability
175       4  0.004026  ability
175       5  0.109566  ability
...     ...       ...      ...
48        5  0.000342     year
48        7  0.000342     year
3490      4  0.000476     yogi
3490      5  0.999189     yogi
3498      3  0.999382      zen

[1532 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 5, 8, 6, 7, 3])

In [23]:
# number optimal of topics
num_topics = 7
# build optimal LDA model
optimal_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics,
                                        alpha=.5,
                                        eta=.1,
                                        random_state=222
                                        )
# filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in optimal_model.print_topics()]

# create topic sorted by 10 most relevent words
topics = [' '.join(t[0:10]) for t in words]


# print most relevent words for each topic
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
man mind power act matter happiness opportunity happen winston churchill

------ Topic 1 ------
thing think proverb dream man john today judge robert thomas

------ Topic 2 ------
change let desire go leader decision tzu forget talk water

------ Topic 3 ------
dyer way amp thought success strength einstein place problem action

------ Topic 4 ------
life body love peace create get control respect napoleon mean

------ Topic 5 ------
heart come try lincoln ask man learn amp franklin courage

------ Topic 6 ------
time world love person look day choice feel moment fall



In [24]:
# Compute Perplexity
## a measure of how good the model is. lower the better
base_perplexity = optimal_model.log_perplexity(corpus)
print('\nPerplexity: ', base_perplexity) 

# Compute Coherence Score
coherence_model = CoherenceModel(model=optimal_model, texts=tweets_cleaned['nouns_verbs'], 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)


Perplexity:  -6.944416918060987

Coherence Score:  0.5703983543287552


In [25]:
# topic distance visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.118773 -0.099551       1        1  15.838854
4      0.223100 -0.020295       2        1  14.859052
1     -0.180366  0.018814       3        1  14.671364
0     -0.276369 -0.075103       4        1  14.271473
5      0.002739 -0.283289       5        1  13.700606
6      0.206018  0.143051       6        1  13.479049
2     -0.093896  0.316373       7        1  13.179601, topic_info=           Term           Freq          Total Category  logprob  loglift
1463       dyer  110188.000000  110188.000000  Default  30.0000  30.0000
78         life   60056.000000   60056.000000  Default  29.0000  29.0000
72        thing   38847.000000   38847.000000  Default  28.0000  28.0000
119        time   39074.000000   39074.000000  Default  27.0000  27.0000
58       change   24886.000000   24886.000000  Default  26.0000  26.0000
...         ...            ...            ...      ...      ...      ...
113        meet    2413.499273    2476.752852   Topic7  -5.0117   2.0006
1027     loving    3023.996800    3972.034628   Topic7  -4.7862   1.7538
206   challenge    2536.115612    2960.864985   Topic7  -4.9621   1.8717
354       power    3928.435218   15541.960180   Topic7  -4.5245   0.6512
2080     defeat    2590.552604    4041.024731   Topic7  -4.9409   1.5819

[463 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
133       4  0.000963    abraham
133       5  0.998989    abraham
27        1  0.000227  abundance
27        2  0.001134  abundance
27        3  0.001361  abundance
...     ...       ...        ...
48        3  0.992067       year
48        4  0.002609       year
48        5  0.000326       year
48        6  0.000326       year
3498      4  0.999659        zen

[1353 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 2, 1, 6, 7, 3])